In [2]:
# Importação de todas as bibliotecas necessárias
import pandas as pd
import json
from datetime import datetime
import os
import matplotlib.pyplot as plt
import seaborn as sns

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


In [ ]:
class SQLtoNoSQLMigrator:
    def __init__(self):
        self.alunos_df = None
        self.cursos_df = None
        self.matriculas_df = None
        self.documentos_alunos = []
        
    def carregar_csvs(self, alunos_path, cursos_path, matriculas_path):
        """Carrega os arquivos CSV com delimitador ;"""
        try:
            print("Carregando arquivos CSV...")
            
            # Carregar com encoding correto para caracteres especiais
            self.alunos_df = pd.read_csv(alunos_path, delimiter=';', encoding='utf-8')
            self.cursos_df = pd.read_csv(cursos_path, delimiter=';', encoding='utf-8')
            self.matriculas_df = pd.read_csv(matriculas_path, delimiter=';', encoding='utf-8')
            
            print("✅ CSVs carregados com sucesso!")
            print(f"📊 Alunos: {len(self.alunos_df)} registros")
            print(f"📚 Cursos: {len(self.cursos_df)} registros")
            print(f"🎓 Matrículas: {len(self.matriculas_df)} registros")
            
            # Mostrar preview dos dados
            self._mostrar_preview()
            
        except Exception as e:
            print(f"❌ Erro ao carregar CSVs: {e}")
            
    def _mostrar_preview(self):
        """Mostra uma prévia dos dados carregados"""
        print("\n--- PRÉVIA DOS DADOS ---")
        print("\n📋 Alunos (primeiras 3 linhas):")
        display(self.alunos_df.head(3))
        
        print("\n📚 Cursos (primeiras 3 linhas):")
        display(self.cursos_df.head(3))
        
        print("\n🎓 Matrículas (primeiras 5 linhas):")
        display(self.matriculas_df.head(5))
    # Continuação da classe SQLtoNoSQLMigrator
    def validar_dados(self):
        """Valida se os dados necessários estão presentes"""
        print("\n🔍 Validando estrutura dos dados...")
        
        # Verificar colunas presentes
        print("📋 Colunas em alunos:", list(self.alunos_df.columns))
        print("📚 Colunas em cursos:", list(self.cursos_df.columns))
        print("🎓 Colunas em matrículas:", list(self.matriculas_df.columns))
        
        # Mapear nomes de colunas
        aluno_id_col = 'id'
        curso_id_col = 'id' 
        matricula_id_col = 'id'
        
        required_mappings = {
            'alunos': [aluno_id_col, 'nome', 'numero_matricula'],
            'cursos': [curso_id_col, 'nome', 'codigo'],
            'matriculas': [matricula_id_col, 'aluno_id', 'curso_id']
        }
        
        for tabela, colunas in required_mappings.items():
            df = getattr(self, f'{tabela}_df')
            missing_cols = [col for col in colunas if col not in df.columns]
            if missing_cols:
                raise ValueError(f"❌ Colunas faltantes em {tabela}: {missing_cols}")
            else:
                print(f"✅ {tabela.capitalize()}: OK")
        
        return aluno_id_col, curso_id_col, matricula_id_col
    
    def analisar_dados(self):
        """Faz uma análise exploratória dos dados"""
        print("\n📊 ANÁLISE EXPLORATÓRIA DOS DADOS")
        
        # Análise de alunos
        print(f"\n👥 ALUNOS:")
        print(f"Total: {len(self.alunos_df)}")
        print(f"Status: {self.alunos_df['status'].value_counts().to_dict()}")
        
        # Análise de cursos
        print(f"\n📚 CURSOS:")
        print(f"Total: {len(self.cursos_df)}")
        print(f"Distribuição de créditos: {self.cursos_df['creditos'].value_counts().sort_index().to_dict()}")
        
        # Análise de matrículas
        print(f"\n🎓 MATRÍCULAS:")
        print(f"Total: {len(self.matriculas_df)}")
        print(f"Status: {self.matriculas_df['status'].value_counts().to_dict()}")
        
        # Matrículas por aluno
        matriculas_por_aluno = self.matriculas_df.groupby('aluno_id').size()
        print(f"\n📈 Matrículas por aluno:")
        print(f"Máximo: {matriculas_por_aluno.max()}")
        print(f"Mínimo: {matriculas_por_aluno.min()}")
        print(f"Média: {matriculas_por_aluno.mean():.2f}")
    # Continuação da classe SQLtoNoSQLMigrator
    def migrar_para_nosql(self):
        """Realiza a migração para estrutura NoSQL"""
        print("\n🔄 INICIANDO MIGRAÇÃO PARA NOSQL...")
        
        aluno_id_col, curso_id_col, matricula_id_col = self.validar_dados()
        self.documentos_alunos = []
        
        # Para cada aluno, buscar suas matrículas
        for index, aluno in self.alunos_df.iterrows():
            aluno_id = aluno[aluno_id_col]
            
            if index % 5 == 0:  # Progresso a cada 5 alunos
                print(f"🔄 Processando aluno {index + 1}/{len(self.alunos_df)}...")
            
            # Buscar matrículas do aluno
            matriculas_aluno = self.matriculas_df[
                self.matriculas_df['aluno_id'] == aluno_id
            ]
            
            matriculas_formatadas = []
            
            for _, matricula in matriculas_aluno.iterrows():
                curso_id = matricula['curso_id']
                
                # Buscar informações do curso
                curso_match = self.cursos_df[self.cursos_df[curso_id_col] == curso_id]
                if not curso_match.empty:
                    curso_info = curso_match.iloc[0]
                    
                    matricula_doc = {
                        'matricula_id': int(matricula[matricula_id_col]),
                        'curso_id': int(curso_id),
                        'data_matricula': matricula.get('data_matricula', ''),
                        'status': matricula.get('status', ''),
                        'nota': float(matricula.get('nota', 0)),
                        'criado_em': matricula.get('criado_em', '')
                    }
                    
                    # Adicionar informações do curso à matrícula
                    matricula_doc['curso_info'] = {
                        'codigo': curso_info.get('codigo', ''),
                        'nome': curso_info['nome'],
                        'descricao': curso_info.get('descricao', ''),
                        'creditos': int(curso_info.get('creditos', 0)),
                        'capacidade': int(curso_info.get('capacidade', 0)),
                        'categoria': self._extrair_categoria(curso_info.get('nome', ''))
                    }
                    
                    matriculas_formatadas.append(matricula_doc)
            
            # Criar documento do aluno
            aluno_doc = {
                'aluno_id': int(aluno_id),
                'numero_matricula': aluno.get('numero_matricula', ''),
                'nome': aluno['nome'],
                'email': aluno.get('email', ''),
                'data_nascimento': aluno.get('data_nascimento', ''),
                'data_ingresso': aluno.get('data_ingresso', ''),
                'status_aluno': aluno.get('status', ''),
                'criado_em': aluno.get('criado_em', ''),
                'matriculas': matriculas_formatadas,
                'total_matriculas': len(matriculas_formatadas),
                'matriculas_ativas': len([m for m in matriculas_formatadas if m.get('status') == 'matriculado']),
                'matriculas_concluidas': len([m for m in matriculas_formatadas if m.get('status') == 'concluido']),
                'matriculas_canceladas': len([m for m in matriculas_formatadas if m.get('status') == 'cancelado'])
            }
            
            self.documentos_alunos.append(aluno_doc)
        
        print(f"✅ Migração concluída: {len(self.documentos_alunos)} documentos criados")
        return self.documentos_alunos
    
    def _extrair_categoria(self, nome_curso):
        """Extrai categoria baseada no nome do curso"""
        nome_curso = nome_curso.lower()
        
        categorias = {
            'Matemática': ['matemática', 'cálculo', 'álgebra', 'estatística'],
            'Programação': ['programação', 'python', 'javascript', 'desenvolvimento', 'banco de dados', 'sql'],
            'Ciências': ['física', 'biologia', 'química'],
            'Tecnologia': ['sistemas', 'redes', 'ia', 'inteligência artificial', 'engenharia de software', 'ux'],
            'Humanas': ['história', 'inglês', 'economia', 'empreendedorismo', 'gestão', 'design', 'projetos']
        }
        
        for categoria, palavras_chave in categorias.items():
            if any(palavra in nome_curso for palavra in palavras_chave):
                return categoria
        
        return 'Geral'
    # Continuação da classe SQLtoNoSQLMigrator
    def gerar_estatisticas(self):
        """Gera estatísticas da migração"""
        if not self.documentos_alunos:
            print("❌ Nenhum documento gerado. Execute a migração primeiro.")
            return
            
        print("\n📊 ESTATÍSTICAS DA MIGRAÇÃO")
        print("=" * 50)
        
        total_alunos = len(self.documentos_alunos)
        total_matriculas = sum(doc['total_matriculas'] for doc in self.documentos_alunos)
        matriculas_ativas = sum(doc['matriculas_ativas'] for doc in self.documentos_alunos)
        matriculas_concluidas = sum(doc['matriculas_concluidas'] for doc in self.documentos_alunos)
        matriculas_canceladas = sum(doc['matriculas_canceladas'] for doc in self.documentos_alunos)
        
        print(f"👥 Total de alunos: {total_alunos}")
        print(f"🎓 Total de matrículas: {total_matriculas}")
        print(f"✅ Matrículas ativas: {matriculas_ativas}")
        print(f"🎯 Matrículas concluídas: {matriculas_concluidas}")
        print(f"❌ Matrículas canceladas: {matriculas_canceladas}")
        print(f"📈 Média de matrículas por aluno: {total_matriculas/total_alunos:.2f}")
        
        # Distribuição por categoria de curso
        categorias = {}
        for doc in self.documentos_alunos:
            for matricula in doc['matriculas']:
                categoria = matricula['curso_info']['categoria']
                categorias[categoria] = categorias.get(categoria, 0) + 1
        
        print(f"\n📚 DISTRIBUIÇÃO POR CATEGORIA:")
        for categoria, count in sorted(categorias.items(), key=lambda x: x[1], reverse=True):
            print(f"  {categoria}: {count} matrículas ({count/total_matriculas*100:.1f}%)")
        
        # Alunos sem matrículas
        alunos_sem_matriculas = len([doc for doc in self.documentos_alunos if doc['total_matriculas'] == 0])
        print(f"\n⚠️  Alunos sem matrículas: {alunos_sem_matriculas}")
        
        self._gerar_graficos(categorias, total_matriculas)
    
    def _gerar_graficos(self, categorias, total_matriculas):
        """Gera gráficos das estatísticas"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        # Gráfico 1: Distribuição de categorias
        labels = list(categorias.keys())
        sizes = list(categorias.values())
        ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
        ax1.set_title('Distribuição de Matrículas por Categoria')
        
        # Gráfico 2: Status das matrículas
        status_data = {
            'Ativas': sum(doc['matriculas_ativas'] for doc in self.documentos_alunos),
            'Concluídas': sum(doc['matriculas_concluidas'] for doc in self.documentos_alunos),
            'Canceladas': sum(doc['matriculas_canceladas'] for doc in self.documentos_alunos)
        }
        ax2.bar(status_data.keys(), status_data.values(), color=['green', 'blue', 'red'])
        ax2.set_title('Status das Matrículas')
        ax2.set_ylabel('Quantidade')
        
        # Gráfico 3: Distribuição de matrículas por aluno
        matriculas_por_aluno = [doc['total_matriculas'] for doc in self.documentos_alunos]
        ax3.hist(matriculas_por_aluno, bins=range(0, max(matriculas_por_aluno) + 2), alpha=0.7, edgecolor='black')
        ax3.set_title('Distribuição de Matrículas por Aluno')
        ax3.set_xlabel('Número de Matrículas')
        ax3.set_ylabel('Número de Alunos')
        
        # Gráfico 4: Status dos alunos
        status_alunos = {}
        for doc in self.documentos_alunos:
            status = doc['status_aluno']
            status_alunos[status] = status_alunos.get(status, 0) + 1
        
        ax4.pie(status_alunos.values(), labels=status_alunos.keys(), autopct='%1.1f%%', startangle=90)
        ax4.set_title('Status dos Alunos')
        
        plt.tight_layout()
        plt.show()
    

In [ ]:
# CÉLULA DE EXECUÇÃO PRINCIPAL

print("🚀 INICIANDO MIGRAÇÃO COMPLETA SQL → NoSQL")
print("=" * 50)

# 1. Instanciar o migrador
migrator = SQLtoNoSQLMigrator()

# 2. Carregar os arquivos CSV
print("\n1️⃣  CARREGANDO ARQUIVOS CSV")
migrator.carregar_csvs(
    alunos_path='alunos.csv',
    cursos_path='cursos.csv', 
    matriculas_path='matricula.csv'
)

# 3. Analisar dados originais
print("\n2️⃣  ANALISANDO DADOS ORIGINAIS")
migrator.analisar_dados()

# 4. Realizar migração
print("\n3️⃣  REALIZANDO MIGRAÇÃO")
documentos = migrator.migrar_para_nosql()

# 5. Gerar estatísticas
print("\n4️⃣  GERANDO ESTATÍSTICAS")
migrator.gerar_estatisticas()

# 6. Mostrar exemplo de documento
print("\n5️⃣  EXEMPLO DE DOCUMENTO GERADO")
migrator.mostrar_exemplo_documento(aluno_id=1)

# 7. Salvar resultados
print("\n6️⃣  SALVANDO RESULTADOS")
migrator.salvar_json('documentos_alunos_completo.json')
migrator.salvar_individual('documentos_individual')

print("\n🎉 MIGRAÇÃO CONCLUÍDA COM SUCESSO!")

In [ ]:
# CÉLULA DE CONSULTAS EXEMPLO

def consultas_exemplo(documentos):
    """Exemplos de consultas que podem ser feitas nos documentos NoSQL"""
    
    print("🔍 EXEMPLOS DE CONSULTAS NOSQL")
    print("=" * 50)
    
    # 1. Alunos com mais matrículas
    print("\n1️⃣  TOP 5 ALUNOS COM MAIS MATRÍCULAS:")
    alunos_mais_matriculas = sorted(documentos, key=lambda x: x['total_matriculas'], reverse=True)[:5]
    for aluno in alunos_mais_matriculas:
        print(f"   {aluno['nome']}: {aluno['total_matriculas']} matrículas")
    
    # 2. Alunos ativos sem matrículas ativas
    print("\n2️⃣  ALUNOS ATIVOS SEM MATRÍCULAS ATIVAS:")
    alunos_ativos_sem_matriculas = [
        aluno for aluno in documentos 
        if aluno['status_aluno'] == 'ativo' and aluno['matriculas_ativas'] == 0
    ]
    for aluno in alunos_ativos_sem_matriculas:
        print(f"   {aluno['nome']} (ID: {aluno['aluno_id']})")
    
    # 3. Buscar alunos por categoria de curso
    print("\n3️⃣  ALUNOS MATRICULADOS EM CURSOS DE PROGRAMAÇÃO:")
    alunos_programacao = []
    for aluno in documentos:
        tem_programacao = any(
            'Programação' in matricula['curso_info']['categoria'] 
            for matricula in aluno['matriculas']
        )
        if tem_programacao:
            alunos_programacao.append(aluno['nome'])
    
    print(f"   Total: {len(alunos_programacao)} alunos")
    print(f"   Exemplos: {', '.join(alunos_programacao[:3])}...")
    
    # 4. Matrículas concluídas com nota acima de 8
    print("\n4️⃣  MATRÍCULAS CONCLUÍDAS COM NOTA ≥ 8.0:")
    boas_notas = []
    for aluno in documentos:
        for matricula in aluno['matriculas']:
            if matricula['status'] == 'concluido' and matricula['nota'] >= 8.0:
                boas_notas.append({
                    'aluno': aluno['nome'],
                    'curso': matricula['curso_info']['nome'],
                    'nota': matricula['nota']
                })
    
    for nota in sorted(boas_notas, key=lambda x: x['nota'], reverse=True)[:5]:
        print(f"   {nota['aluno']} - {nota['curso']}: {nota['nota']}")

# Executar consultas exemplo
if migrator.documentos_alunos:
    consultas_exemplo(migrator.documentos_alunos)